In [14]:
from keras.layers import Conv2D,UpSampling2D,InputLayer
from keras.models import Sequential,load_model
import tensorflow as tf
import numpy as np
from skimage.color import rgb2lab,lab2rgb,rgb2gray
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img,img_to_array
from skimage.io import imsave 
import os
from keras.callbacks import TensorBoard

**DATA** **PRE-PROCESSING**

In [287]:
X = []
Y=[]
for filename in os.listdir('dataset800/Train'):
    X.append(rgb2lab(img_to_array(load_img('dataset800/Train/'+filename)))[:,:,0])
    Y.append(rgb2lab(img_to_array(load_img('dataset800/Train/'+filename)))[:,:,1:])
X = np.array(X, dtype=float)
Y=np.array(Y,dtype=float)
X=X.reshape(len(X),128,128,1)
Y=Y.reshape(len(Y),128,128,2)
X=X/255
Y=Y/128

In [15]:
X = []
for filename in os.listdir('dataset800/Train/'):
    X.append(img_to_array(load_img('dataset800/Train/'+filename)))
X = np.array(X, dtype=float)
X=1.0/255*X
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(X, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

**Encoding** **layers**

In [16]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))

**Decoding** **Layers**

In [17]:
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))

**Compiling**

In [18]:
model.compile(optimizer='rmsprop',loss='mse')

**Training**

In [20]:
tensorboard=tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(image_a_b_gen(batch_size),callbacks=[tensorboard],epochs=100,steps_per_epoch=10)

Epoch 1/100


C:\Users\sujay\AppData\Local\Temp\ipykernel_13360\2637906542.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(image_a_b_gen(batch_size),callbacks=[tensorboard],epochs=100,steps_per_epoch=10)


10/10 [==============================] - 10s 964ms/step - loss: 0.8267
Epoch 2/100
10/10 [==============================] - 9s 916ms/step - loss: 0.8253
Epoch 3/100
10/10 [==============================] - 9s 935ms/step - loss: 0.8290
Epoch 4/100
10/10 [==============================] - 9s 925ms/step - loss: 0.8262
Epoch 5/100
10/10 [==============================] - 9s 919ms/step - loss: 0.8245
Epoch 6/100
10/10 [==============================] - 9s 935ms/step - loss: 0.8280
Epoch 7/100
10/10 [==============================] - 9s 914ms/step - loss: 0.8322
Epoch 8/100
10/10 [==============================] - 10s 953ms/step - loss: 0.8276
Epoch 9/100
10/10 [==============================] - 9s 928ms/step - loss: 0.8309
Epoch 10/100
10/10 [==============================] - 9s 927ms/step - loss: 0.8248
Epoch 11/100
10/10 [==============================] - 9s 940ms/step - loss: 0.8296
Epoch 12/100
10/10 [==============================] - 9s 915ms/step - loss: 0.8308
Epoch 13/100
10/10 [===

10/10 [==============================] - 9s 933ms/step - loss: 0.0015
Epoch 100/100
10/10 [==============================] - 10s 963ms/step - loss: 0.0015


In [9]:

print(model.evaluate(X,Y,batch_size=1))

ValueError: in user code:

    File "C:\Users\sujay\anaconda3\lib\site-packages\keras\engine\training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\sujay\anaconda3\lib\site-packages\keras\engine\training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\sujay\anaconda3\lib\site-packages\keras\engine\training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\sujay\anaconda3\lib\site-packages\keras\engine\training.py", line 1667, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\sujay\anaconda3\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\sujay\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\s

**Saving** **model**

In [24]:
model.save('Image_colorizer_model.h5')

**Preprocessing Testing images**

In [21]:
Z=[]
for filename in os.listdir('dataset/Test'):
    Z.append(rgb2lab(img_to_array(load_img('dataset/Test/'+ filename)))[:,:,0])
Z=np.array(Z,dtype=float)
Z=Z/255
Z=Z.reshape(len(Z),256,256,1)

**Running tests**

In [22]:
#model=load_model('Image_colorizer_model.h5')
output=model.predict(Z)
output=output*128

1/1 [==============================] - 0s 485ms/step


**Saving images**

In [25]:
for i in range(len(output)):
  cur=np.zeros((256,256,3))
  cur[:,:,0]=Z[i][:,:,0]
  cur[:,:,1:]=output[i]
  imsave('results/img_'+ str(i)+'.png',lab2rgb(cur))

C:\Users\sujay\AppData\Local\Temp\ipykernel_13360\1409828496.py:5: UserWarning: new_result/img_0.png is a low contrast image
  imsave('new_result/img_'+ str(i)+'.png',lab2rgb(cur))
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\sujay\AppData\Local\Temp\ipykernel_13360\1409828496.py:5: UserWarning: new_result/img_2.png is a low contrast image
  imsave('new_result/img_'+ str(i)+'.png',lab2rgb(cur))
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\sujay\AppData\Local\Temp\ipykernel_13360\1409828496.py:5: UserWarning: new_result/img_4.png is a low contrast image
  imsave('new_result/img_'+ str(i)+'.

In [63]:
model=load_model('Image_colorizer_model.h5')
Z=[]
for filename in os.listdir('dataset/Test'):
    Z.append(rgb2lab(img_to_array(load_img('dataset/Test/'+ filename)))[:,:,0])
Z=np.array(Z,dtype=float)
Z=Z/255
Z=Z.reshape(len(Z),256,256,1)
output=model.predict(Z)
output=output*128
for i in range(len(output)):
  cur=np.zeros((256,256,3))
  cur[:,:,0]=Z[i][:,:,0]
  cur[:,:,1:]=output[i]
  imsave('new_result/img_'+ str(i)+'.png',lab2rgb(cur))

1/1 [==============================] - 1s 513ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
